<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>


<center> Nathan Barrett, Benjamin Feder, Joseph Chappell </center>

# Unsupervised Machine Learning

Sometimes we do not have a clear outcome variable, but nevertheless want to explore and discover any inherent groupings or patterns in the data. Unsupervised machine learning methods can help tackle these problems. Clustering is the most common unsupervised machine learning technique, but you might also be aware of principal components analysis (PCA) or neural networks implementations such as self-organizing maps (SOM). This notebook will provide an introduction to unsupervised machine learning through a clustering example.

## Introduction to Clustering

Clustering is used to group data points together that are similar to each other. Optimally, a given clustering method will produce groupings with high intra-cluster (within) similarity and low inter-cluster (between) similarity. Clustering algorithms typically require a distance or similarity metric to generate clusters. They take a dataset and a distance metric (and sometimes additional parameters), and they generate clusters based on that distance metric. The most common distance metric used is Euclidean distance, but other commonly-used metrics are Manhattan, Minkowski, Chebyshev, Cosine, Hamming, Pearson, and Mahalanobis.

Most clustering algorithms also require the user to specify the number of clusters (or some other parameter that indirectly determines the number of clusters) in advance as a parameter. This is often difficult to do a priori and typically makes clustering an iterative and interactive task. Another aspect of clustering that makes it interactive is often the difficulty in automatically evaluating the quality of the clusters. While various analytical clustering metrics have been developed, the best clustering is task-dependent and thus must be evaluated by the user. There may be different clusterings that can be generated with the same data. You can imagine clustering similar news stories based on the topic content, on the writing style or sentiment. The right set of clusters depends on the user and the task at hand. Clustering is therefore typically used for exploring the data, generating clusters, exploring the clusters, and then rerunning the clustering method with different parameters or modifying the clusters (by splitting or merging the previous set of clusters). Interpreting a cluster can be nontrivial: you can look at the centroid of a cluster, look at frequency distributions of different features (and compare them to the prior distribution of each feature), or other aspects.

Here, we will focus on **K-Means clustering** (*k* defines the number of clusters), which is considered to be the most commonly used clustering method. The algorithm works as follows:
1. Select *k* (the number of clusters you want to generate).
2. Initialize by selecting k points as centroids of the *k* clusters. This is typically done by selecting k points uniformly at random.
3. Assign each point a cluster according to the nearest centroid.
4. Recalculate cluster centroids based on the assignment in **(3)** as the mean of all data points belonging to that cluster.
5. Repeat **(3)** and **(4)** until convergence.

The algorithm stops or "converges" when the assignments do not change from one iteration to the next. The final set of clusters, however, depends on the starting points. If initialized differently, it is possible that different clusters are obtained. One common practical trick is to run *k*-means several times, each with different (random) starting points. The *k*-means algorithm is fast, simple, and easy to use, and is often a good first clustering algorithm to try and see if it fits your needs. When the mean of the data points cannot be computed, a related method called *K-medoids* can be used.

### Learning Objectives

This notebook demonstrates using *k*-means clustering to better understand Tennessee's labor market in 2017. We've already developed a handful of employer measures in a supplemental notebook. We will try a few different values of *k* to see how we can best understand the labor market by looking for differentiation between each of the clusters.

## Import Packages and Set Up


The main R package that we will use for clustering is called `cluster`. We also import all our usual packages for database connection and data manipulation/visualization.

In [ ]:
# Database interaction imports
library(odbc, warn.conflicts=F, quietly=T)

# For data manipulation/visualization
library(tidyverse, warn.conflicts=F, quietly=T)

# For faster date conversions
library(lubridate, warn.conflicts=F, quietly=T)

# Use percent() function
library(scales, warn.conflicts=F, quietly=T)

# clustering
library(cluster, warn.conflicts=F, quietly=T)

In [ ]:
# Connect to the server
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

In [ ]:
# Code adjusting overall graph attributes

# For easier reading, increase base font size
theme_set(theme_gray(base_size = 16))
# Adjust repr.plot.width and repr.plot.height to change the size of graphs
options(repr.plot.width = 12, repr.plot.height = 8)

## 1. Read in the Data

We will read in a table`employer_yearly_agg`, which contains characteristics of Tennessee's labor market from 2015-2019. This mimics the years included in the employment outcomes analyses in the data exploration and visualization notebooks. For more information as to how this table was created, please refer to the Supplemental notebook ["Supplemental_Employer_Measures.ipynb."](Supplemental/Supplemental_Employer_Measures.ipynb)

`employer_yearly_agg` contains variables tracking the following characteristics on a quarterly basis, with the average (`avg_`) quuarterly values reported within each year. The measures can be broken up within three separate categories:

Stability:

    - Number of full quarter employees
    - Separation growth rate

Opportunity:

    - Hiring growth rate
    - Number of employees
    - Employment growth rate

Job Quality:

    - Total payroll
    - Earnings per employee at the 25th percentile
    - Earnings per employee at the 75th percentile
    - Total payroll for full quarter employees
    - Average earnings per employee
    - Average earnings per full quarter employee

    
In addition, the variable `num_quarters` tracks the number of quarters in which the associated `empr_nbr` existed in Tennessee's Unemployment Insurance wage records with at least five employees in the given year.

> Note: For example, the variable `avg_num_employed` contains the average number of quarterly employees for a given year/employer combination.

In [ ]:
# read into R
qry <- "
select *
from tr_tn_2021.dbo.employer_yearly_agg
"

emp <- dbGetQuery(con, qry)

# see employers
head(emp)

In [ ]:
# see all available columns
names(emp)

To get a glimpse at the labor market in a specific calendar year, we will subset our data frame to include information only for one year: 2017.

> Note: It is possible to cluster on all employers in all years at the same time as well—just keep in mind that there are a different set of assumptions that are prevalent with that decision.

In [ ]:
# Subset a dataframe by rows from 2017

emp <- emp %>%
    filter(year == 2017)

In [ ]:
# Check that we only have 2017
emp %>%
    distinct(year)

### Clean the Data

We need to remove the `empr_nbr`, `year`, and `num_quarters` variables from our data frame since these features do not provide any explanatory power for our k-means algorithm. Additionally, k-means algorithms only work properly with continuous features. This is because k-means calculates its distance measure using euclidean distance, which is the distance between each data point and the centroid of a cluster. It is hard to assign positions for categorical variables in the euclidean space.

> There are more sophisticated clustering algorithms that do not use Euclidean distances and thus allow categorical variables in the model. If you are interested in them, you can take a look at the functions `kmodes` and `gower.dist` in R - you will need to download their respective libraries first.

In [ ]:
# Remove empr_nbr, year, and num_quarters
emp_ml <- emp %>%
    select(-c(empr_nbr, year, num_quarters))

In [ ]:
# Check data type of all variables - make sure all of them are numeric
str(emp_ml)

**It is important that we consider scaling these features** before we compute *k*-means clustering, especially if the metrics are on a variety of numerical scales. Let's see if they are.

In [ ]:
# Get descriptions of each variable using "summary" function
summary(emp_ml)

We can see that we have variables on different numerical scales - we can scale them using `scale()` function on our data frame `emp_ml`. Before we do so, though, let's convert all variables to `numeric` types, as the `scale()` function will return `Inf` and `-Inf` for the variables that are of `integer64` type.

In [ ]:
# convert all numeric variables to numeric type otherwise integer64 won't scale
emp_ml_num <- emp_ml %>%
    sapply(as.numeric)

In [ ]:
# Scale the features
emp_ml_scale <- scale(emp_ml_num)

# View first rows after scaling
emp_ml_scale %>% 
   head()

Before running a clustering algorithm, we need to make sure that there are no missing values. Here we will use `na.omit()` to remove all rows with any NA values. If an employer has missing information in any of the columns, a row will be dropped.

> Note that you should **never remove data** if possible - in a real world setting you would likely want to fill any missing data with an imputation method or a baseline assumption. We will discuss missing data during the Inference lecture.

In [ ]:
# Check number of rows (where each row is a unique employer/year combination)
nrow(emp_ml_scale)

In [ ]:
# na.omit will remove any rows with any NA values
emp_ml_scale <- na.omit(emp_ml_scale)

In [ ]:
# Check number of rows after dropping rows with any NA values
# see that there is no missing data
nrow(emp_ml_scale)

## 2. Choose the Number of Clusters, *K*

Running a *k*-means model is simple: we just need to use `kmeans()` and choose the number of clusters (called `centers`). What number should we choose? Here, we have 11 features, so it is hard to visualize the data and decide the proper number by using our eyes. Let's start with a small number, such as 3, and see how the results look.  

Because *k*-means clustering will generate different results (due to different starting points), we will set a seed so that the work in this notebook can be reproducible using the `set.seed()`. To get the same results, you must use the same seed before running the clustering algorithm every time. Luckily, if you set the same seed as your collaborators and are running the same *k*-means algorithm, you will see the same results, even if you are working in different environments, such as jupyter notebooks.

### k = 3

In [ ]:
# Initialize the model and run on emp_ml
set.seed(1)
k3 <- kmeans(emp_ml_scale, centers=3, nstart=20)

> `nstart` specifies a number of initial configurations and reports on the best one - an optimal number is usually somewhere between 20 and 50. (See more information in the Resources section - Professor Steorts, Duke University).

In [ ]:
# see structure
str(k3)

`kmeans` function returns the following components, with the most useful for us now as:
- `cluster` - an integer indicating a cluster to which each point is allocated
- `centers` - a matrix of cluster centers
- `totss` - the total sum of squares
- `withinss` - vector of within-cluster sum of squares, one component per cluster.
- `tot.withinss` - total within-cluster sum of squares, i.e. `sum(withinss)`
- `betweenss` - the between-cluster sum of squares, i.e. `totss-tot.withinss`
- `size` - the number of points in each cluster

Let's check the size of each cluster:

In [ ]:
k3$size

We can see that most of the employers are concentrated in cluster 1. In the perfect world, we would want them to be distributed more evenly across clusters, but in some cases, it may make sense that they wouldn't. Most importantly, we are looking for **high intra-cluster similarity and low inter-cluster similarity**.

Are there major differences in the characteristics of employers in each cluster?

We can take a look at basic descriptives of the employers in these clusters by adding our clustering results to the original dataframe, `emp`, and call this dataframe `frame_3`. This will allow us to start to get a sense of some of the important differences across the clusters in the hopes of categorizing each cluster as a separate "type" of employer relative to those in other clusters.

In [ ]:
# remove missing values
emp <- na.omit(emp) 

# add cluster number to the original dataframe
frame_3 <- emp %>%                     
    mutate(k3.cluster = k3$cluster)  

# remove empr_nbr, year, and num_quarters columns
frame_3 <- frame_3 %>%
    select(-c(empr_nbr, year, num_quarters))

# summarize and add in sizes of each cluster
frame_3 %>%
    group_by(k3.cluster) %>%
    summarize_all("mean") %>%
    mutate(
        size = k3$size
    )

In general, we can see that our biggest cluster, cluster 1, contains relatively small employers that pay their employees lower wages and have less turnover. Cluster 3 also has relatively small employers, and on average, they pay their employees less and have higher turnover rates than those in the other two clusters. In contrast, cluster 2 contains some of the larger employers in the state that tend to pay better and have higher job stability.

## Evaluate clusters

One simple way to characterize the resulting clusters is to compare the summary stats between key variables of interest. 

We can also visualize the differences between the clusters in more detail by finding the mean and standard deviation for the following variables: `avg_avg_earnings`, `avg_bottom_25_pctile`, and `avg_top_75_pctile`. First, we will need to convert our summarized data frame of the mean and standard deviations for the desired variables into a long format, with each variable/cluster combination corresponding to a distinct row.

In [ ]:
# Save results with mean to a dataframe
frame_3_mean <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(k3.cluster, avg_avg_earnings, avg_bottom_25_pctile, avg_top_75_pctile) %>%
    summarize_all("mean") %>%
    mutate(
        avg_avg_earnings = as.numeric(avg_avg_earnings)
        ) %>%
    pivot_longer(-k3.cluster, names_to = "variable", values_to = "mean")

# Save results with standard deviation to a dataframe
frame_3_sd <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(k3.cluster, avg_avg_earnings, avg_bottom_25_pctile, avg_top_75_pctile) %>%
    summarize_all("sd") %>%
    mutate(
        avg_avg_earnings = as.numeric(avg_avg_earnings)
    ) %>%
    pivot_longer(-k3.cluster, names_to = "variable", values_to = "sd") %>%
    select(-c(k3.cluster, variable))

# Bind two dataframes together
df <- cbind(frame_3_mean,frame_3_sd)

df

Now we can use this data frame to visualize the means and standard deviations in our 3 clusters for these 3 variables: `avg_avg_earnings`, `avg_bottom_25_pctile`, and `avg_top_75_pctile` using a bar plot, separating the mean and standard deviations values for each variable into different grids.

In [ ]:
ggplot(df, aes(x=k3.cluster, y=mean, fill=k3.cluster)) +
    geom_bar(stat="identity", position = position_dodge()) +   # plot bars for the mean values
    geom_errorbar(aes(ymax= mean + sd, ymin = mean),            # add standard deviation bars
                  width=.2,
                  position = position_dodge(.9)) +
    facet_grid(. ~ variable) +                                  # plot by 3 variables of interest
    ggtitle("Overlap in mean + SD between clusters 1 and 3") +  # add title
    xlab("Clusters") +                                          # add label for x-axis
    ylab("Mean") +                                              # add label for y-axis
    theme(text = element_text(size=16),                         # increase text font
          axis.text.x = element_text(size=18, face="bold"),     # increase text font on x-axis and make it bold
          legend.position = "none")                             # remove legend

### Visualization functions

We can also create a function to facilitate visualizing different columns in a similar way. The function takes the mean, standard deviation, and title of the visualization as its arguments.

In [ ]:
# Save it to a dataframe
frame_3_mean_sd <- frame_3 %>%
    group_by(k3.cluster) %>%
    select(k3.cluster, avg_avg_earnings, avg_bottom_25_pctile, avg_top_75_pctile) %>%
    summarise_all(list(mean = "mean", sd = "sd"))

# Visualize average earnings by cluster
viz <- function(mean, sd, title) {
    ggplot(frame_3_mean_sd, aes(x=k3.cluster, y=mean, fill=k3.cluster)) +
    geom_bar(position = position_dodge(), stat="identity", fill="gray") +
    geom_errorbar(aes(ymax= mean + sd, ymin = mean),
                  width=.2,
                  position = position_dodge(.9)) +
    ggtitle(title) +
    xlab("Clusters") +
    ylab("Mean") +
    theme(text = element_text(size=16),
          axis.text.x = element_text(size=18, face="bold"),
          legend.position = "none")
    }

In [ ]:
viz(as.numeric(frame_3_mean_sd$avg_avg_earnings_mean), as.numeric(frame_3_mean_sd$avg_avg_earnings_sd), "The Average Quarterly Earnings Are Much Higher in Cluster 2")

In [ ]:
viz(as.numeric(frame_3_mean_sd$avg_bottom_25_pctile_mean), as.numeric(frame_3_mean_sd$avg_bottom_25_pctile_sd), "The Average 25th Percentile of Quarterly Earnings are Much Higher in Cluster 2")

In [ ]:
viz(
    as.numeric(frame_3_mean_sd$avg_top_75_pctile_mean), 
    as.numeric(frame_3_mean_sd$avg_top_75_pctile_sd), 
    "The Average 75th Percentile of Quarterly Earnings are Much Higher in Cluster 2 \n But Have More Spread"
)

### Compare industries

In theory, you can also compare clusters by looking at the most common industries within each cluster by matching the employers in the original data frame (`emp`) back to the UI wage records for the given year and `empr_nbr`. When examing industries by cluster, be aware that some employers may have multiple associated NAICS codes within a calendar year. If you run into this situation where there are multiple non-999999 industry codes, you will need to devise a set of consistent decision rules to apply across the entire table.

## Selecting *k*

How do we know if we chose an optimal number of clusters to describe our data?

### Elbow method

We can use the *Elbow method* as an input in selecting the optimal cluster number. Recall that *k*-means starts with k random cluster centers (centroids), assigns each data point to the closest centroid, and calculates the distances between each point and the centroid. Then it moves the positions of the centroids and repeats the previous steps until there is convergence. In the *Elbow method*, we try different k values and calculate the sum of squared errors (`SSE`) after the model converges. Then we plot all the `SSE` by K in a line-chart. The line-chart should resemble an arm.

In [ ]:
set.seed(1)

# function to compute total within-cluster sum of square
wss <- function(k) {
    kmeans(emp_ml_scale, centers=k, nstart = 1)$tot.withinss
}

# compute and plot wss for k =1 to k = 15
k.values <- 1:15

# extract wss values for each k
wss_values <- map_dbl(k.values, wss)

# plot the resulting SSE for each value of k
plot(k.values, wss_values, 
    type = "b", pch=19, frame=FALSE,
    xlab = "Number of clusters K", 
    ylab = "Total within-clusters sum of squares")

We can see that SSE decreases as we increase k. Here, it decreases faster when k is small. As k increases, the reduction in SSE becomes smaller. We try to choose the number around the inflection point, where the change in SSE becomes negligible, indicating that there is little room to improve the model by increasing k (the bend in the elbow). On our graph, the elbow curve begins to flatten around k = 4.

> Note: The jaggedness in the curve is due to the default `nstart` value being `1`. However, due to the size of the data, if you increase `nstart` in the function above, it may not converge based on the number of clusters.

Let's run the model with 4 clusters.

In [ ]:
set.seed(1)
k4 <- kmeans(emp_ml_scale, centers = 4, nstart = 20)
k4$size

In [ ]:
# add cluster number to the original dataframe
frame_4 <- emp %>%                     
    mutate(k4.cluster = k4$cluster)  

# remove empr_nbr, year, and num_quarters columns
frame_4 <- frame_4 %>%
    select(-c(empr_nbr, year, num_quarters))

# summarize and add in sizes of each cluster
frame_4 %>%
    group_by(k4.cluster) %>%
    summarize_all("mean") %>%
    mutate(
        size = k4$size
    )

The large employers are now even more separated into Cluster 3. These employers tend to pay reasonably well, with low turnover rates, but they do not pay as well as those in Cluster 2. Clusters 2 and 4 contain employers of similar size, but the difference in pay is quite glaring between the two. Cluster 1 consists primarily of much smaller employers with high turnover rates and low pay.

Which clustering results - `frame_3` or `frame_4` - do you prefer? Do you think it could be optimal to choose more clusters?

In summary, in clustering there is no single right answer - every time we run a different number of clusters, interesting patterns about our data can be exposed. However, what we do want to know is *whether the clusters that we find represent true subgroups in our data*. This could be a crucial input toward choosing the right number of clusters. (See more information on additional methods for selecting `k` in the Resources section - Professor Steorts, Duke University).

Experiment with different numbers of clusters in the Checkpoint 1 below - given knowledge about Tennessee's labor market in 2017, which number of clusters makes most sense to you?

<font color=red><h3> Checkpoint 1: Run a K-Means clustering model </h3></font> 

1. Take a look again at the elbow curve, which number(s) do you think is (are) optimal?

2. Choose a cluster number that you think is best (other than 3 or 4). Use `kmeans()` to run a k-means clustering model with the number you choose. Save your results and features in `frame_k`. 

3. Compare your results with the results we got previously. Do you find any differences? Are the results improved, in your opinion?

Hint: in the Elbow method graph, it looks like 12 could be another optimal cluster - you can try with 12 clusters and see the differences, but remember: to achieve convergence, you’ll probably need to set nstart=1.

In [ ]:
# run clustering algorithm using different k

set.seed(1)
__ <- kmeans(emp_ml_scale, centers = ___, nstart = __)
__$size

In [ ]:
# explore differences between clusters

__ <- emp %>%                     
    mutate(__.cluster = __$cluster)  

# remove empr_nbr, year, and num_quarters columns
___ <- ___ %>%
    select(-c(empr_nbr, year, num_quarters))

# summarize and add in sizes of each cluster
___ %>%
    group_by(___) %>%
    summarize_all("mean") %>%
    mutate(
        size = __$size
    )

### Cohort's Employers

In this section we will take a look at our cohort's employers, and identify the clusters they belong to based on the `frame_4` clustering results. Additionally, we will look at our cohort's distribution across the four clusters and compare their employment outcomes.

> We will need to subset `nb_cohort_wages_link` to just jobs in 2017 in order to match these clusters.

In [ ]:
# read earnings of cohort into R subset to 2017 jobs
qry = "
select *
from tr_tn_2021.dbo.nb_cohort_wages_link
where year(job_date) = '2017'
"
df_wages <- dbGetQuery(con, qry)

In this notebook, we will not subset to the dominant employer for each `hashed_ssn` in `df_wages`.

In [ ]:
# add cluster number to the original dataframe
frame_4 <- emp %>%                     
    mutate(k4.cluster = k4$cluster)  

# Join wages table with frame_4 clustering results
df_wages_clus <- df_wages %>%
    inner_join(frame_4, by='empr_nbr')

First, let's look at the number of employers that employed at least one individual in our original cohort in 2017 by cluster.

In [ ]:
# Group by clusters and find number of unique employers in each cluster
df_wages_clus %>%
    group_by(k4.cluster) %>%
    summarise(emp_cohort = n_distinct(empr_nbr))

We can also compare that to the percentage of all employers in our clusters that hired at least one individual in our cohort in the calendar year.

In [ ]:
# Get number of unique employers per cluster in the full dataframe (all employers)
frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(emp_all = n_distinct(empr_nbr))

Instead of scrolling between these two outputs to compare the proportion of employers in each cluster that employed at least one individual in our cohort, we can combine them into one data frame using `inner_join()`.

In [ ]:
# Save cohort and all employers dataframes

cohort_emp <- df_wages_clus %>%
    group_by(k4.cluster) %>%
    summarise(emp_cohort = n_distinct(empr_nbr))

emp_all <- frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(emp_all = n_distinct(empr_nbr))

# Join cohort employers with all employers, andd find percentage
cohort_emp %>%
    inner_join(emp_all, by = 'k4.cluster') %>%
    mutate(percentage = (emp_cohort / emp_all) * 100)

As a reminder, one limitation of our original employers file is that it doesn't include quarterly information for employers with less than 5 employees in a specific quarter. We can see that outside of the employers in Cluster 3, the majority of employers in other clusters did not employ anyone in our cohort in 2017

We can also compare average earnings of our cohort by cluster with average earnings of all employees in each cluster:

In [ ]:
# Calculate average earnings for TN graduates by cluster
# also see number of individuals within each cluster
df_wages_clus %>%
    group_by(k4.cluster) %>%
    summarise(
        mean_earnings_cohort = mean(wge_amt),
        num_individuals = n_distinct(SSN)
    )

In [ ]:
# Calculate average earnings for all employees by cluster
frame_4 %>%
    group_by(k4.cluster) %>%
    summarise(mean_earnings_all = mean(avg_avg_earnings))

Note that among our 2015-16 graduating cohort, those who worked for employers in clusters 2 earned more, on average. We can see similar trends in average quarterly wages amongst all employees and those in our cohort acrosss the clusters, except that those in the cohort made less on average, relative to the average employee. Additionally, We can see that out of those in the cohort who were employed in Tennessee in 2017, the majority were employed by employers in Clusters 3 and 4.

### Breakdown by Most Common Majors

We can also see how recipients of the most common degrees within the graduating cohort placed into these clusters.  Recall we have the most common majors for the overall cohort saved as the csv file `common_major.csv`.

<font color=red> Note that you need to change the directory in read_csv() statements below. Replace ". ." with your username.</font>

In [ ]:
# most common majors
common_major <- read_csv("U:\\..\\TN Training\\Results\\common_major.csv")

common_major

We can limit employment information from `df_wages_clus` to just those receiving degrees in the most common majors with a `filter()` command. Afterwards, we can count the number of individuals within each  major/cluster combination then finding the proportion of individuals with each degree that are employed by employers in the four clusters.

In [ ]:
# find proportions employment by cluster within the most common majors
prop_by_cip_cluster <- df_wages_clus %>% 
    filter(CIP_Family %in% common_major$CIP_Family) %>%
    group_by(CIP_Family, k4.cluster) %>%
    summarize(
        n_inds = n_distinct(SSN)
    ) %>%
    ungroup() %>%
    group_by(CIP_Family) %>%
    mutate(
        prop_by_cluster = n_inds / sum(n_inds)
    ) %>%
    ungroup() 

head(prop_by_cip_cluster)

Since there are a lot of rows (one for each `CIP_Family`/`k4.cluster` combination), we can more easily discern the breakdown of employment by cluster within each major grouping with a simple bar plot.

In [ ]:
prop_by_cip_cluster %>%
    ggplot(aes(x=word(CIP_Family, 1), y = prop_by_cluster, fill=as.character(k4.cluster))) +
    geom_col(position=position_dodge()) +
    labs(
        x = "Major",
        y = "Proportion by Cluster",
        fill = "Cluster"
    ) +
    ylim(0, 1)

As somewhat expected given the sorting of the overall cohort into primarily employers in cluster 4, all degree recipients were most commonly employed by employers in cluster 4 in Tennessee in 2017. However, you can note that those receiving degrees in Engineering and Computer and Information Sciences were more likely to be employed by employers in Cluster 2, the cluster with the highest average quarterly pay, than those receiving degrees in the other three majors.

<font color=red><h3> Checkpoint 2: Cohort's Employers </h3></font> 

Repeat this exploration of employer clusters for the sample you created in Checkpoint 1. Which clusters hire the greatest proportion of our original cohort? Which pay the most?

## Resources
- UC Business Analytics R Programming Guide: https://uc-r.github.io/kmeans_clustering
- Rebecca Steorts, Assistant Professor, Duke University, Department of Statistical Science, Data Mining and Machine Learning course: https://github.com/resteorts/data-mine/tree/master/lectures_2018/10-unsupervise/10-kmeans.pdf